# SQL with Python Reference Guide 5
# The JOIN family of operators
## (Justin M. Olds)
Based on Stanford SQL course: https://lagunita.stanford.edu/courses/DB/SQL/SelfPaced/info

---
**JOIN operators overview** 
* INNER JOIN ON *condition*
* NATURAL JOIN
* INNER JOIN USING(*attributes*)
* LEFT|RIGHT|FULL OUTER JOIN


In [4]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("class.db")
c = conn.cursor()

---
### Tables and Insert code hidden below (same as before--college admissions data)

In [9]:
c.execute('DROP TABLE IF EXISTS College')
c.execute('DROP TABLE IF EXISTS Student') 
c.execute('DROP TABLE IF EXISTS Apply') 

c.execute('CREATE TABLE College(cName TEXT, state TEXT, enrollment INT)')
c.execute('CREATE TABLE Student(sID INT, sName TEXT, GPA REAL, sizeHS INT)')
c.execute('CREATE TABLE Apply(sID INT, cName TEXT, major TEXT, decision TEXT)')
conn.commit()

In [10]:
c.execute('DELETE FROM Student')
c.execute('DELETE FROM College')
c.execute('DELETE FROM Apply')

c.execute("INSERT INTO Student VALUES (123, 'Amy', 3.9, 1000)")
c.execute("INSERT INTO Student values (234, 'Bob', 3.6, 1500)")
c.execute("INSERT INTO Student values (345, 'Craig', 3.5, 500)")
c.execute("INSERT INTO Student values (456, 'Doris', 3.9, 1000)")
c.execute("INSERT INTO Student values (567, 'Edward', 2.9, 2000)")
c.execute("INSERT INTO Student values (678, 'Fay', 3.8, 200)")
c.execute("INSERT INTO Student values (789, 'Gary', 3.4, 800)")
c.execute("INSERT INTO Student values (987, 'Helen', 3.7, 800)")
c.execute("INSERT INTO Student values (876, 'Irene', 3.9, 400)")
c.execute("INSERT INTO Student values (765, 'Jay', 2.9, 1500)")
c.execute("INSERT INTO Student values (654, 'Amy', 3.9, 1000)")
c.execute("INSERT INTO Student values (543, 'Craig', 3.4, 2000)")

c.execute("INSERT INTO College values ('Stanford', 'CA', 15000)")
c.execute("INSERT INTO College values ('Berkeley', 'CA', 36000)")
c.execute("INSERT INTO College values ('MIT', 'MA', 10000)")
c.execute("INSERT INTO College values ('Cornell', 'NY', 21000)")

c.execute("INSERT INTO Apply values (123, 'Stanford', 'CS', 'Y')")
c.execute("INSERT INTO Apply values (123, 'Stanford', 'EE', 'N')")
c.execute("INSERT INTO Apply values (123, 'Berkeley', 'CS', 'Y')")
c.execute("INSERT INTO Apply values (123, 'Cornell', 'EE', 'Y')")
c.execute("INSERT INTO Apply values (234, 'Berkeley', 'biology', 'N')")
c.execute("INSERT INTO Apply values (345, 'MIT', 'bioengineering', 'Y')")
c.execute("INSERT INTO Apply values (345, 'Cornell', 'bioengineering', 'N')")
c.execute("INSERT INTO Apply values (345, 'Cornell', 'CS', 'Y')")
c.execute("INSERT INTO Apply values (345, 'Cornell', 'EE', 'N')")
c.execute("INSERT INTO Apply values (678, 'Stanford', 'history', 'Y')")
c.execute("INSERT INTO Apply values (987, 'Stanford', 'CS', 'Y')")
c.execute("INSERT INTO Apply values (987, 'Berkeley', 'CS', 'Y')")
c.execute("INSERT INTO Apply values (876, 'Stanford', 'CS', 'N')")
c.execute("INSERT INTO Apply values (876, 'MIT', 'biology', 'Y')")
c.execute("INSERT INTO Apply values (876, 'MIT', 'marine biology', 'N')")
c.execute("INSERT INTO Apply values (765, 'Stanford', 'history', 'Y')")
c.execute("INSERT INTO Apply values (765, 'Cornell', 'history', 'N')")
c.execute("INSERT INTO Apply values (765, 'Cornell', 'psychology', 'Y')")
c.execute("INSERT INTO Apply values (543, 'MIT', 'CS', 'N')")
conn.commit()


---
### Start with simple query (used previously)

The statement below finds the Names and majors of all students (from student table) that applied (same students in the apply table). 

In [11]:
df = pd.read_sql_query("""
    SELECT DISTINCT sName, major
    FROM Student, Apply
    WHERE Student.sID = Apply.sID
""", conn)
df

,sName,major
0,Amy,CS
1,Amy,EE
2,Bob,biology
3,Craig,CS
4,Craig,EE
5,Craig,bioengineering
6,Fay,history
7,Helen,CS
8,Irene,CS
9,Irene,biology


### Using the INNER JOIN (also can be written as simply JOIN) operator
The above result **joins** two tables and the same result can be obtained by using an INNER JOIN operator in the following way. 

In [12]:
df = pd.read_sql_query("""
    SELECT DISTINCT sName, major
    FROM Student INNER JOIN Apply
    WHERE Student.sID = Apply.sID
""", conn)
df

,sName,major
0,Amy,CS
1,Amy,EE
2,Bob,biology
3,Craig,CS
4,Craig,EE
5,Craig,bioengineering
6,Fay,history
7,Helen,CS
8,Irene,CS
9,Irene,biology


Now, let's see what happens when we have a join operation with additional conditions besides the one that references a common attribute of found on both tables. 

In [13]:
df = pd.read_sql_query("""
    SELECT sName, GPA
    FROM Student, Apply 
    WHERE Student.sID = Apply.sID
        AND sizeHS < 1000 
        AND major = 'CS'
        AND cName = 'Stanford'
""", conn)
df

,sName,GPA
0,Helen,3.7
1,Irene,3.9


Rewritten with the JOIN operator: 

In [14]:
df = pd.read_sql_query("""
    SELECT sName, GPA
    FROM Student INNER JOIN Apply 
    ON Student.sID = Apply.sID
        WHERE sizeHS < 1000 
        AND major = 'CS'
        AND cName = 'Stanford'
""", conn)
df

,sName,GPA
0,Helen,3.7
1,Irene,3.9


**Style note:** Queries like the one above can have conditions following an ON keyword or WHERE keyword. It is useful to list condtions that are specifically related to the how tables are joining immediately following the ON keyword and to list conditions that are not following the WHERE keyword like done above. 

### Joining three tables: 
The following query is similar provides a join of the Student and Apply table like before, but also joins the College table to include the enrollment number corresponding to each college that a particular student applied to.


In [15]:
df = pd.read_sql_query("""
    SELECT Apply.sID, sName, GPA, Apply.cName, enrollment
    FROM Apply, Student, College
    WHERE Apply.sID = Student.sID AND Apply.cName = College.cName
""", conn)
df

,sID,sName,GPA,cName,enrollment
0,123,Amy,3.9,Stanford,15000
1,123,Amy,3.9,Stanford,15000
2,123,Amy,3.9,Berkeley,36000
3,123,Amy,3.9,Cornell,21000
4,234,Bob,3.6,Berkeley,36000
5,345,Craig,3.5,MIT,10000
6,345,Craig,3.5,Cornell,21000
7,345,Craig,3.5,Cornell,21000
8,345,Craig,3.5,Cornell,21000
9,678,Fay,3.8,Stanford,15000


Rewritten using the JOIN operators: 

In [16]:
df = pd.read_sql_query("""
    SELECT Apply.sID, sName, GPA, Apply.cName, enrollment
    FROM Apply INNER JOIN Student INNER JOIN College
    ON Apply.sID = Student.sID AND Apply.cName = College.cName
""", conn)
df

,sID,sName,GPA,cName,enrollment
0,123,Amy,3.9,Stanford,15000
1,123,Amy,3.9,Stanford,15000
2,123,Amy,3.9,Berkeley,36000
3,123,Amy,3.9,Cornell,21000
4,234,Bob,3.6,Berkeley,36000
5,345,Craig,3.5,MIT,10000
6,345,Craig,3.5,Cornell,21000
7,345,Craig,3.5,Cornell,21000
8,345,Craig,3.5,Cornell,21000
9,678,Fay,3.8,Stanford,15000


**Style note:** Parenthesis can be used to make the joining conditions more readily apparent -- This binary join formatting is actually required for some SQL systems (e.g., PostRisk). See query below: 

In [18]:
df = pd.read_sql_query("""
    SELECT Apply.sID, sName, GPA, Apply.cName, enrollment
    FROM (Apply INNER JOIN Student ON Apply.sID = Student.sID)
        INNER JOIN College ON Apply.cName = College.cName
""", conn)
df

,sID,sName,GPA,cName,enrollment
0,123,Amy,3.9,Stanford,15000
1,123,Amy,3.9,Stanford,15000
2,123,Amy,3.9,Berkeley,36000
3,123,Amy,3.9,Cornell,21000
4,234,Bob,3.6,Berkeley,36000
5,345,Craig,3.5,MIT,10000
6,345,Craig,3.5,Cornell,21000
7,345,Craig,3.5,Cornell,21000
8,345,Craig,3.5,Cornell,21000
9,678,Fay,3.8,Stanford,15000


### Other JOIN operators (NATURAL and USING(*attribute*))

The examples above have used the INNER JOIN operator but were also the equivalent to NATURAL JOINs because NATURAL JOINs take two relations that have column names in common and performs a cross product that only keeps the tuples with the same values for those common attribute names. Before this was expressed explicitly (e.g., Apply.sID = Student.sID). Replacing INNER JOIN with NATURAL JOIN in the above select statments will 'naturally' find the relations that the two tables have in common and return the rows with matching values on those shared attributes (e.g., sID). See below: 

In [19]:
df = pd.read_sql_query("""
    SELECT DISTINCT sName, major
    FROM Student NATURAL JOIN Apply
""", conn)
df

,sName,major
0,Amy,CS
1,Amy,EE
2,Bob,biology
3,Craig,CS
4,Craig,EE
5,Craig,bioengineering
6,Fay,history
7,Helen,CS
8,Irene,CS
9,Irene,biology


NATURAL JOINs will also 'naturally' eliminate matching columns. For example, the query below returns results with one of the redundant sID columns deleted. 

In [20]:
df = pd.read_sql_query("""
    SELECT *
    FROM Student NATURAL JOIN Apply
""", conn)
df

,sID,sName,GPA,sizeHS,cName,major,decision
0,123,Amy,3.9,1000,Berkeley,CS,Y
1,123,Amy,3.9,1000,Cornell,EE,Y
2,123,Amy,3.9,1000,Stanford,CS,Y
3,123,Amy,3.9,1000,Stanford,EE,N
4,234,Bob,3.6,1500,Berkeley,biology,N
5,345,Craig,3.5,500,Cornell,CS,Y
6,345,Craig,3.5,500,Cornell,EE,N
7,345,Craig,3.5,500,Cornell,bioengineering,N
8,345,Craig,3.5,500,MIT,bioengineering,Y
9,678,Fay,3.8,200,Stanford,history,Y


### USING(*attribute*) clause is highly prefered over NATURAL JOIN because it keeps the joining attributes explicit.

In realistic applications there can be hundreds of attributes that might possibly match under the hood of a NATURAL JOIN query. Thus, the USING clause is recommended to keep the join explicit. 



In [21]:
df = pd.read_sql_query("""
    SELECT sName, GPA
    FROM Student INNER JOIN Apply USING(sID)
    WHERE sizeHS < 1000 and major = 'CS' and cName = 'Stanford'
""", conn)
df

,sName,GPA
0,Helen,3.7
1,Irene,3.9


### Using the JOIN operator when there is more than one instance of the same relation.
(e.g., Student S1, Student S2)
Rather than having S1.GPA = S2.GPA within the WHERE clause the following query JOINs the duplicate relations such that values of GPA match. 

NOTE: With most SQL systems a USING clause does not with with an ON clause within the same statement. This is not a problem because ON can be replaced with WHERE. 

In [22]:
df = pd.read_sql_query("""
    SELECT S1.sID, S1.sName, S1.GPA, S2.sID, S2.sName, S2.GPA
    FROM Student S1 INNER JOIN Student S2 USING(GPA)
    WHERE S1.sID < S2.sID
""", conn)
df

,sID,sName,GPA,sID,sName,GPA
0,123,Amy,3.9,456,Doris,3.9
1,123,Amy,3.9,654,Amy,3.9
2,123,Amy,3.9,876,Irene,3.9
3,456,Doris,3.9,654,Amy,3.9
4,456,Doris,3.9,876,Irene,3.9
5,567,Edward,2.9,765,Jay,2.9
6,654,Amy,3.9,876,Irene,3.9
7,543,Craig,3.4,789,Gary,3.4


### OUTER JOIN 

Start of with an INNER JOIN for some attributes for students taken from the Student and Apply tables.

In [25]:
df = pd.read_sql_query("""
    SELECT sName, sID, cName, major
    FROM Student INNER JOIN Apply USING(sID)
""", conn)
df

,sName,sID,cName,major
0,Amy,123,Berkeley,CS
1,Amy,123,Cornell,EE
2,Amy,123,Stanford,CS
3,Amy,123,Stanford,EE
4,Bob,234,Berkeley,biology
5,Craig,345,Cornell,CS
6,Craig,345,Cornell,EE
7,Craig,345,Cornell,bioengineering
8,Craig,345,MIT,bioengineering
9,Fay,678,Stanford,history


What if we wanted to include the information for students that haven't yet applied anywhere (are in the Student table but not the Apply table)? To accomplish this we could change the above query to use a LEFT OUTER JOIN. These "dandling" tuples will have NULL/None values for any fields 

In [31]:
df = pd.read_sql_query("""
    SELECT sName, sID, cName, major
    FROM Student LEFT OUTER JOIN Apply USING(sID)
""", conn)
df

,sName,sID,cName,major
0,Amy,123,Berkeley,CS
1,Amy,123,Cornell,EE
2,Amy,123,Stanford,CS
3,Amy,123,Stanford,EE
4,Bob,234,Berkeley,biology
5,Craig,345,Cornell,CS
6,Craig,345,Cornell,EE
7,Craig,345,Cornell,bioengineering
8,Craig,345,MIT,bioengineering
9,Doris,456,None,None


For demonstration: rewriting an OUTER JOIN without using the OUTER JOIN operator. 

The following query combines the results of two SELECT statements. The second SELECT statement queries the dangling Student tuples that are not found (via sID) in both the Student and Apply.

In [28]:
df = pd.read_sql_query("""
    SELECT sName, Student.sID, cName, major
    FROM Student, Apply
    WHERE Student.sID = Apply.sID
    UNION
    SELECT sName, sID, NULL, NULL
    from Student
    WHERE sID NOT IN (SELECT sID from Apply)
""", conn)
df

,sName,sID,cName,major
0,Amy,123,Berkeley,CS
1,Amy,123,Cornell,EE
2,Amy,123,Stanford,CS
3,Amy,123,Stanford,EE
4,Amy,654,None,None
5,Bob,234,Berkeley,biology
6,Craig,345,Cornell,CS
7,Craig,345,Cornell,EE
8,Craig,345,Cornell,bioengineering
9,Craig,345,MIT,bioengineering


RIGHT OUTER JOIN can be used to easily change a query to include tuples from the table entered on the right side of the join clause that do not match. 

---
### FULL OUTER JOIN
This includes all dangling tuples from both tables. 

**IMPORTANT NOTE:** Both RIGHT OUTER JOIN and FULL OUTER JOIN are not supported with sqlite but can be expressed in other ways.

Expressing a FULL OUTER JOIN without the JOIN operator:

In [32]:
df = pd.read_sql_query("""
    SELECT sName, Student.sID, cName, major
    FROM Student, Apply 
    WHERE Student.sID = Apply.sID
    UNION
    SELECT sName, sID, NULL, NULL
    FROM Student
    WHERE sID NOT IN (SELECT sID from Apply)
    UNION
    SELECT NULL, sID, cName, major
    FROM Apply
    WHERE sID NOT IN (SELECT sID FROM STUDENT)
""", conn)
df

,sName,sID,cName,major
0,Amy,123,Berkeley,CS
1,Amy,123,Cornell,EE
2,Amy,123,Stanford,CS
3,Amy,123,Stanford,EE
4,Amy,654,None,None
5,Bob,234,Berkeley,biology
6,Craig,345,Cornell,CS
7,Craig,345,Cornell,EE
8,Craig,345,Cornell,bioengineering
9,Craig,345,MIT,bioengineering
